In [3]:
import requests
import json

        start: ALWAYS provide an appropriate line number to insert the 
        added code. This field should NEVER be empty. Ensure the code is placed appropriately in the file. For example,
        if the user wants to add a method to a class, add the method in the appropriate location.
        If the user requests to add code, this header should be the starting line 
        to insert the code. If the user requests to modify code, it should be the starting
        line of the existing code to be replaced by the generated code.
        
        stop: This header should ONLY be populated if the user requests to modify code.
        In this case, provide the last line number to replace the newly generated code with.
        For example, if lines 21-30 should be replaced, the stop will be 30.
        As a reminder, you should ONLY respond in JSON format. There should be no other output besides the headers described.
        Ensure your output is a well-formed JSON.

                type: this should be 'add' if the generated code is to be added into the 
        file, and 'modify' if the code is to replace existing code in the file.

In [38]:

# Define the endpoint URL
url = "http://localhost:11434/api/chat"

# Define the headers and payload
headers = {
    "Content-Type": "application/json"
}

i = '''The indentation should match that
        of the existing code. For example, adding a new method to a class should start with 1 level of indentation,
        and adding code to a method within a class should start with 2 levels.'''

# Initialize conversation history
conversation_history = [
    {
        'role': 'system',
        'content': '''You are an expert programmer that helps to add or modify Python code 
        based on the user request, with concise explanations. Don't be too verbose. 
        Respond in JSON format to a user prompt with the headers code, explanation, type, modify-range.
        
        code: return the well-formed python code generated according to the user request.
        Do NOT reproduce the entire file. Only the requested portion by the user.
        The code should be in JSON format, nested inside the output json with each header as a line number.
        The line numbers should be determined by where the code should be placed in the file.
        For example, a method that should be added to line 21 should start with '21' as the first header of the code json.
        Ensure that the code is indented, formatted, and placed correctly. The indentation should match that
        of the existing code.

        explanation: provide a brief explanation in plain text of what this code does and how it works.

        type: this should be 'ADD' if the generated code is to be added into the 
        file, and 'MODIFY' if the code is to replace existing code in the file.
        
        modify-range: If type is MODIFY, provide the minimal line numbers to be deleted in
        the original file and replaced with the new code for the current modification only. 
        If type is ADD, leave null.
        
        Inputted context code is formatted in JSON format, with the header 
        corresponding to the current line number.

        Your output format should be the following:
        {
        'code': 
            {
                '1': '...',
                '2': '...',
                ...
            },
        'explanation': '...',
        'type': 'ADD or MODIFY'
        'modify-range': [<Start Line Num>, <End Line Num>]
        }
        '''
    },
    {
        'role': 'system',
        'content': 'The current file for the user is in the next message.'
    },
    {
        'role': 'user',
        'content': '''
{
    "1": "class BankAccount:",
    "2": "    def __init__(self, account_number, account_holder):",
    "3": "        self.account_number = account_number",
    "4": "        self.account_holder = account_holder",
    "5": "        self.balance = 0.0",
    "6": "",
    "7": "    def deposit(self, amount):",
    "8": "        if amount > 0:",
    "9": "            self.balance += amount",
    "10": "            print(f\"${amount} deposited. New balance is ${self.balance}\")",
    "11": "        else:",
    "12": "            print(\"Deposit amount must be positive.\")",
    "13": "",
    "14": "    def withdraw(self, amount):",
    "15": "        if 0 < amount <= self.balance:",
    "16": "            self.balance -= amount",
    "17": "            print(f\"${amount} withdrawn. New balance is ${self.balance}\")",
    "18": "        else:",
    "19": "            print(\"Insufficient funds or invalid withdrawal amount.\")",
    "20": "",
    "21": "    def get_balance(self):",
    "22": "        return self.balance",
    "23": "",
    "24": "# Example of creating a BankAccount object",
    "25": "account = BankAccount(\"123456789\", \"John Doe\")",
    "26": "account.deposit(1000)",
    "27": "account.withdraw(500)",
    "28": "print(account.get_balance())"
}

'''
    }
]


# Function to send the message and update the conversation history
def send_message(message, model="llama3.2"):
    global conversation_history
    conversation_history.append({
        "role": "user",
        "content": message,
        "format": 'json'
    })
    payload = {
        "model": model,
        "messages": conversation_history,
        "stream": False
    }
    response = requests.post(url, json=payload)
    if response.status_code == 200:
        response_json = response.json()
        print(response_json)
        content = response_json.get('message', {}).get('content', '')
        conversation_history.append({
            "role": "assistant",
            "content": content
        })
        print(content)
    else:
        print(response.json())
        print(f"Request failed with status code {response.status_code}")


In [39]:
# Example usage
send_message('Add a method in the class called transfer to the BankAccount class to transfer money from one account to another.')
send_message('Add another method in the class called zero balance to the BankAccount class to zero the balance of an account.')
send_message('Great! Now modify the zero balance method to print that the balance in the users account is now zero')

{'model': 'llama3.2', 'created_at': '2024-11-13T01:20:15.0556498Z', 'message': {'role': 'assistant', 'content': '{\n  "code": {\n    "21": "    def transfer(self, amount, account_to_transfer_from):",\n    "22": "        if 0 < amount <= self.balance:",\n    "23": "            if account_to_transfer_from.balance >= amount:",\n    "24": "                account_to_transfer_from.balance -= amount",\n    "25": "                self.balance += amount",\n    "26": "                print(f\'${amount} transferred from ${account_to_transfer_from.account_number} to ${self.account_number}. New balances are ${account_to_transfer_from.balance} and ${self.balance}\')",\n    "27": "            else:",\n    "28": "                print(\'Insufficient funds in the source account.\')"\n    },\n  "explanation": "This method allows users to transfer money from one BankAccount object to another.",\n  "type": "ADD",\n  "modify-range": [null, null]\n}'}, 'done_reason': 'stop', 'done': True, 'total_duration':